In [1]:
from utils import time_op
import impmeas as imp

In [4]:
# tests the model counting capabilities of SharpSAT-TD and Buddy

n,m,k = 8, 10, 7
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

imp.formulas.set_pmc_solver()
solver = imp.formulas.get_pmc_solver()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

imp.formulas.set_buddy_context(vars)
f = time_op(imp.BuddyNode.parse)(formula)
satcount_bdd = time_op(f.expectation)() * 2**n
print(f"BDD satcount = {satcount_bdd}")

[00007.6518 ms / 0.0077 s / 0.0001 min] satcount
GPMC satcount = 232
[00116.1497 ms / 0.1161 s / 0.0019 min] parse
[00000.0074 ms / 0.0000 s / 0.0000 min] expectation
BDD satcount = 232.0


In [7]:
n,clauses_to_variables,k = 17, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [8]:
import impmeas.bdds as bdd 
imp.formulas.set_buddy_context(vars)

x = "x1"
f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[01203.7730 ms / 1.2038 s / 0.0201 min] parse
bdd size = 2839
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7fd9f697e0b0> ===
k=0 d result=0.1877 max increase possible=0.4061 
k=1 d result=0.1915 max increase possible=0.1431 
k=2 d result=0.1150 max increase possible=0.0211 
k=3 d result=0.0204 max increase possible=0.0006 
k=4 d result=0.0006 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5152, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[13049.1862 ms / 13.0492 s / 0.2175 min] blame
[00000.5543 ms / 0.0006 s / 0.0000 min] influence
influence 0.18773, 0.51517 <= blame


In [10]:
imp.formulas.set_pmc_solver()
solver = imp.formulas.get_pmc_solver()

f = time_op(imp.Formula.parse)(formula)
x = "x1"
infl = (time_op)(imp.influence)(f, x)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[01022.3842 ms / 1.0224 s / 0.0170 min] parse
[00307.8694 ms / 0.3079 s / 0.0051 min] influence
=== COMPUTING BLAME for x1 in Formula with size 3191 ===
k=0 size of cnf: 3557 d result=0.1877 max increase possible=0.4061 
k=1 size of cnf: 3616 d result=0.1915 max increase possible=0.1431 
k=2 size of cnf: 3671 d result=0.1150 max increase possible=0.0211 
k=3 size of cnf: 3722 d result=0.0204 max increase possible=0.0006 
k=4 size of cnf: 3769 d result=0.0006 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5152, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[21226.6393 ms / 21.2266 s / 0.3538 min] blame
influence 0.18773, 0.51517 <= blame


In [11]:

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"

f = time_op(imp.Table.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"{blame:.5f} <= blame")
infl = time_op(imp.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x))(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(imp.dominating_cgm)(f)
upsilon = time_op(imp.rectifying_cgm)(f)

print("="*100)
imp.formulas.set_buddy_context(vars)

f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")


[00743.5420 ms / 0.7435 s / 0.0124 min] parse
[01795.8853 ms / 1.7959 s / 0.0299 min] blame
0.59508 <= blame
[00013.4203 ms / 0.0134 s / 0.0002 min] influence
influence = 0.29102
[06412.5905 ms / 6.4126 s / 0.1069 min] <lambda>
hammer = 0.07600
[00750.4482 ms / 0.7504 s / 0.0125 min] dominating_cgm
[00596.4086 ms / 0.5964 s / 0.0099 min] rectifying_cgm
[00433.0618 ms / 0.4331 s / 0.0072 min] parse
bdd size = 91
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7fd9f5f09120> ===
k=0 d result=0.2910 max increase possible=0.3545 
k=1 d result=0.2148 max increase possible=0.0931 
k=2 d result=0.0778 max increase possible=0.0115 
k=3 d result=0.0112 max increase possible=0.0002 
k=4 d result=0.0002 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5951, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[00084.7430 ms / 0.0847 s / 0.0014 min] blame
[00000.0637 ms / 0.0001 s / 0.0000 min] influence
influence 0.29102,